# Stablecoin Premiums — Batch Markets Export

Batch-fetch P2P stablecoin quotes across multiple fiat markets, compute premiums/spreads versus FX reference rates, and export results to CSV (and optionally Parquet).

Prerequisites:
- Create and activate a virtual environment
- `pip install -r requirements.txt`
- Copy `.env.example` to `.env` and fill credentials (XE API credentials are required for FX)
- Never commit `.env` to version control

Outputs:
- CSV file saved in `../data/` (relative to this notebook) with a UTC timestamp
- Optional Parquet file (if `pyarrow` or `fastparquet` is installed and enabled below)


In [ ]:
# Optional: load a local .env (best-effort). If python-dotenv is not installed, this is skipped.
try:
    from dotenv import load_dotenv  # type: ignore
    load_dotenv()
except Exception:
    pass

from stablecoin_premiums import setup_logging
setup_logging("INFO")
print("Environment loaded (if present). Logging initialized.")

## Parameters

Configure the batch run: list of fiats, asset, reference fiat, and export settings. If `DEFAULT_FIATS` is configured in your environment, you can leave `FIATS` empty to use those defaults instead.

In [ ]:
from typing import List

# --- User-editable parameters ---
FIATS: List[str] = [
    "MXN", "BRL", "ARS", "COP", "CLP", "PEN"
]  # If you leave this empty list, DEFAULT_FIATS from env will be used (if set)

ASSET = "USDT"           # Stablecoin to query on P2P
    REF_FIAT = "USD"         # Reference FX fiat for mid-rate
MIN_VALID_ADS = None      # Require at least this many valid P2P ads; None to skip check
DECIMALS = 2              # Round computed outputs; set to None for full precision
SLEEP_BETWEEN = 0.0       # Seconds to sleep between fiat requests (0.0 for none)

OUTPUT_DIR = "../data"   # Relative to this notebook; path is gitignored by default
WRITE_PARQUET = False     # If True, attempt to write a Parquet file (requires pyarrow or fastparquet)

# If FIATS is empty, attempt to use DEFAULT_FIATS from environment config
from stablecoin_premiums.config import DEFAULT_FIATS as ENV_DEFAULT_FIATS
if not FIATS:
    FIATS = ENV_DEFAULT_FIATS[:] if ENV_DEFAULT_FIATS else ["MXN"]

print("Batch parameters:")
print({
    "FIATS": FIATS,
    "ASSET": ASSET,
    "REF_FIAT": REF_FIAT,
    "MIN_VALID_ADS": MIN_VALID_ADS,
    "DECIMALS": DECIMALS,
    "OUTPUT_DIR": OUTPUT_DIR,
    "WRITE_PARQUET": WRITE_PARQUET,
})

## Imports & Helpers

We use the Binance P2P client for quotes, the XE client for FX, and `compute_premiums` to calculate premium metrics. The helper function below processes a single fiat into a normalized record for export.

In [ ]:
import os
import time
import json
from datetime import datetime, timezone
from typing import Dict, Any

import pandas as pd

from stablecoin_premiums.clients.binance import average_price
from stablecoin_premiums.clients.xe import fetch_fx_rate
from stablecoin_premiums.compute import compute_premiums

def process_fiat(
    fiat: str,
    *,
    asset: str,
    ref_fiat: str,
    min_valid_ads=None,
    decimals=None,
) -> Dict[str, Any]:
    """
    Fetch BUY/SELL quotes (Binance P2P), FX mid (XE), and compute premiums/spreads for one fiat.

    Returns a dict with fields:
      - fiat, asset, ref_fiat
      - buy_rate, sell_rate
      - fx_bid, fx_ask (if available)
      - stablecoin_sell_premium, stablecoin_buy_premium, stablecoin_buy_sell_spread (if computable)
      - status: "ok" or an error marker
    """
    record: Dict[str, Any] = {
        "timestamp_utc": datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ"),
        "fiat": fiat,
        "asset": asset,
        "ref_fiat": ref_fiat,
    }

    buy = average_price(fiat, asset, "BUY", min_valid_ads=min_valid_ads)
    sell = average_price(fiat, asset, "SELL", min_valid_ads=min_valid_ads)
    fx = fetch_fx_rate(fiat, ref_fiat)

    record["buy_rate"] = buy
    record["sell_rate"] = sell

    if buy is None or sell is None:
        record["status"] = "insufficient_p2p_data"
        return record

    if not fx:
        record["status"] = "insufficient_fx_data"
        return record

    record["fx_bid"] = fx["bid"]
    record["fx_ask"] = fx["ask"]

    try:
        premiums = compute_premiums(
            sell_rate=sell,
            buy_rate=buy,
            fx_bid=fx["bid"],
            fx_ask=fx["ask"],
            decimals=decimals,
        )
        record.update(premiums.to_dict())
        record["status"] = "ok"
    except Exception as exc:
        record["status"] = f"compute_error: {exc}"

    return record

print("Helper ready: process_fiat(fiat, asset=ASSET, ref_fiat=REF_FIAT)")

## Batch Run

We iterate over the configured fiats, collect rows, and build a DataFrame. Set `SLEEP_BETWEEN` above to throttle requests if needed (avoid rate limits).

In [ ]:
rows = []
for idx, fiat in enumerate(FIATS, start=1):
    print(f"[{idx}/{len(FIATS)}] Processing {fiat} …")
    row = process_fiat(
        fiat,
        asset=ASSET,
        ref_fiat=REF_FIAT,
        min_valid_ads=MIN_VALID_ADS,
        decimals=DECIMALS,
    )
    rows.append(row)
    if SLEEP_BETWEEN and SLEEP_BETWEEN > 0:
        time.sleep(SLEEP_BETWEEN)
print("Done.")

df = pd.DataFrame(rows)
df

## Export to CSV (and optionally Parquet)

Exports results with a UTC timestamped filename into `OUTPUT_DIR`. Parquet export requires `pyarrow` or `fastparquet` when enabled.

In [ ]:
os.makedirs(OUTPUT_DIR, exist_ok=True)
ts = datetime.now(timezone.utc).strftime("%Y%m%dT%H%M%SZ")
csv_path = os.path.join(OUTPUT_DIR, f"premiums_batch_{ts}.csv")
df.to_csv(csv_path, index=False)

parquet_path = None
if WRITE_PARQUET:
    try:
        parquet_path = os.path.join(OUTPUT_DIR, f"premiums_batch_{ts}.parquet")
        df.to_parquet(parquet_path, index=False)
    except Exception as exc:
        print(f"Parquet export skipped (missing engine or error): {exc}")

print("CSV written:", csv_path)
if parquet_path:
    print("Parquet written:", parquet_path)

df.head()

## Status Summary

Quick overview of which fiats computed successfully and which encountered data issues (e.g., insufficient P2P ads or FX).

In [ ]:
summary = df.groupby("status", dropna=False)["fiat"].count().rename("count").reset_index()
summary.sort_values("count", ascending=False).reset_index(drop=True)

## JSON Snapshot (Optional)

Pretty-print in-memory results as JSON for quick copy/paste or auxiliary storage. This does not write a file by default—use the CSV/Parquet export above for persistent outputs.

In [ ]:
print(json.dumps(rows, indent=2, ensure_ascii=False))